In [23]:
from sklearn.decomposition import PCA

import numpy as np
import os
from PIL import Image

import joblib

%matplotlib inline

#이미지 입력
#소고기 원본사진
path = './testpicture/1/24.jpg'
def evaluate_grade(path):
    
    image_pil = Image.open(path)
    image = np.array(image_pil)

    xaverage=image.shape[0]
    yaverage=image.shape[1]
    #한변의 길이의 절반
    halfpicsize=150

    #target이미지가 원본사진을 이탈하는것 방지
    if yaverage<halfpicsize+10:
        yaverage=halfpicsize+10
    elif yaverage>len(image)-(halfpicsize+10):
        yaverage=len(image)-(halfpicsize+10)

    if xaverage<(halfpicsize+10):
        xaverage=(halfpicsize+10)
    elif xaverage>len(image[0])-(halfpicsize+10):
        xaverage=len(image[0])-(halfpicsize+10)


    #300*300 cutting    
    cutimage=image_pil.crop((xaverage-halfpicsize,yaverage-halfpicsize,xaverage+halfpicsize,yaverage+halfpicsize))
    #300*300=>50*50 resizing
    resizedcutimage=np.array(cutimage.resize((50,50),Image.LANCZOS))
    #3차원=>2차원
    resizedcutimage=np.concatenate(resizedcutimage)
    #2차원=>1차원
    resizedcutimage=np.concatenate(resizedcutimage)

    #target 데이터 저장
    j=[]
    j.append(resizedcutimage)
    
     
    #300*300 target cutting기준
    #K-neighbor모델을 위한 pca모델
    pca31=joblib.load('./pca_component31.pkl')
    
    #target data 변환
    #K-neighbor 전용
    k=pca31.transform(j)
    j=[]
    
   
    
    #모델 불러오기
    loaded_kn15_model=joblib.load( './pca_knn_neighbor15.pkl')

    #print("KN-15모델 예측")
    #print(loaded_kn15_model.predict(k))
    #print(loaded_kn15_model.classes_)
    #print(loaded_kn15_model.predict_proba(k))
    
    result=[]
    
    for i in range(5):
        newdict={}
        newdict["className"]=loaded_kn15_model.classes_[i]
        newdict["probability"]=loaded_kn15_model.predict_proba(k)[0][i]
        result.append(newdict)
    return result
    
evaluate_grade(path)


[{'className': '1', 'probability': 0.0},
 {'className': '1+', 'probability': 0.8050855160600559},
 {'className': '1++', 'probability': 0.0},
 {'className': '2', 'probability': 0.1949144839399442},
 {'className': '3', 'probability': 0.0}]